In [4]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

df = pd.read_csv('youtube-comments.csv')
df.shape

(892, 7)

In [5]:
df.head()

,id,video_id,comment_id,text_display,text_original,is_spam,checked
0,2,1z4ughaHFj4,UgyOqRur7XVuSZsC-uN4AaABAg,"<a href=""https://youtu.be/9XHO4W7_4UI"">https:/...",https://youtu.be/9XHO4W7_4UI,1,1
1,5,1z4ughaHFj4,UgwKDIi_qJwGP2U0rOx4AaABAg,ලස්සන්‍යි බන්...වීඩියො එක බලන්නෙ නෙ සින්දුව අහ...,ලස්සන්‍යි බන්...වීඩියො එක බලන්නෙ නෙ සින්දුව අහ...,0,1
2,6,1z4ughaHFj4,Ugyeu3pV4VglihhGiOh4AaABAg,????පව් කරලා වැඩක් නැ සිල් අරන්.....මම් යන්නම ...,????පව් කරලා වැඩක් නැ සිල් අරන්.....මම් යන්නම ...,0,1
3,14,1z4ughaHFj4,UgzhtHRdH0xJUVZ8pvp4AaABAg,රැල්ලට ගහගෙන යන්න එපා අයියේ. වෙනස් දෙයක් කරන්න...,රැල්ලට ගහගෙන යන්න එපා අයියේ. වෙනස් දෙයක් කරන්න...,0,1
4,20,1z4ughaHFj4,UgwMWsprBrAjYKlyF1x4AaABAg,"වොයිස් එකට හදපන්කෝ හොද සිංදුවක්,, අඩන සිංදු නැ...","වොයිස් එකට හදපන්කෝ හොද සිංදුවක්,, අඩන සිංදු නැ...",0,1


In [6]:
df['has_link'] = 0
df['has_subscribe'] = 0
df['has_email'] = 0
df['has_phone'] = 0

word_list = ['subscribe', 'subcribe', 'subcrib', 'සබ්ස්']

for index, row in df.iterrows():
    if any(re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', str(row['text_original']))):
        df.at[index, 'has_link'] = 1
    
    if any(re.findall(r'|'.join(word_list), str(row['text_original']), re.IGNORECASE)):
        df.at[index, 'has_subscribe'] = 1
        
    if any(re.findall(r'(^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', str(row['text_original']))):
        df.at[index, 'has_email'] = 1
        
    if any(re.findall(r'[0-9]{7,}', str(row['text_original']))):
        df.at[index, 'has_phone'] = 1

df.head()

,id,video_id,comment_id,text_display,text_original,is_spam,checked,has_link,has_subscribe,has_email,has_phone
0,2,1z4ughaHFj4,UgyOqRur7XVuSZsC-uN4AaABAg,"<a href=""https://youtu.be/9XHO4W7_4UI"">https:/...",https://youtu.be/9XHO4W7_4UI,1,1,1,0,0,0
1,5,1z4ughaHFj4,UgwKDIi_qJwGP2U0rOx4AaABAg,ලස්සන්‍යි බන්...වීඩියො එක බලන්නෙ නෙ සින්දුව අහ...,ලස්සන්‍යි බන්...වීඩියො එක බලන්නෙ නෙ සින්දුව අහ...,0,1,0,0,0,0
2,6,1z4ughaHFj4,Ugyeu3pV4VglihhGiOh4AaABAg,????පව් කරලා වැඩක් නැ සිල් අරන්.....මම් යන්නම ...,????පව් කරලා වැඩක් නැ සිල් අරන්.....මම් යන්නම ...,0,1,0,0,0,0
3,14,1z4ughaHFj4,UgzhtHRdH0xJUVZ8pvp4AaABAg,රැල්ලට ගහගෙන යන්න එපා අයියේ. වෙනස් දෙයක් කරන්න...,රැල්ලට ගහගෙන යන්න එපා අයියේ. වෙනස් දෙයක් කරන්න...,0,1,0,0,0,0
4,20,1z4ughaHFj4,UgwMWsprBrAjYKlyF1x4AaABAg,"වොයිස් එකට හදපන්කෝ හොද සිංදුවක්,, අඩන සිංදු නැ...","වොයිස් එකට හදපන්කෝ හොද සිංදුවක්,, අඩන සිංදු නැ...",0,1,0,0,0,0


In [7]:
print(len(df[df.has_subscribe==1]))
print(len(df[df.has_link==1]))

24
56


In [8]:
df.loc[df['is_spam']==1].head()

,id,video_id,comment_id,text_display,text_original,is_spam,checked,has_link,has_subscribe,has_email,has_phone
0,2,1z4ughaHFj4,UgyOqRur7XVuSZsC-uN4AaABAg,"<a href=""https://youtu.be/9XHO4W7_4UI"">https:/...",https://youtu.be/9XHO4W7_4UI,1,1,1,0,0,0
11,34,1z4ughaHFj4,UgxqFuEF4dq-O-co29l4AaABAg,එ බන් උබලත් කැමතිද ගෙදට්ට වෙලාම ඔන්ලයින් සල්ලි...,එ බන් උබලත් කැමතිද ගෙදට්ට වෙලාම ඔන්ලයින් සල්ලි...,1,1,1,0,0,0
17,47,1z4ughaHFj4,UgyQK1eOsykUCv9M-vt4AaABAg,මේකෙ 8d audio එක ඕනෙනම් මේකට like කරන්න,මේකෙ 8d audio එක ඕනෙනම් මේකට like කරන්න,1,1,0,0,0,0
18,48,1z4ughaHFj4,UgwJTTbEBXvWaclbKzB4AaABAg,"<a href=""https://gokano.com/ref/IBmbOvZSMRoJ0x...",https://gokano.com/ref/IBmbOvZSMRoJ0x9ZwAFQXIvM,1,1,1,0,0,0
22,56,1z4ughaHFj4,UgzUXOYNVxbGLLTjw5x4AaABAg,"<a href=""https://youtu.be/aXrFxHE8s90"">https:/...",https://youtu.be/aXrFxHE8s90,1,1,1,0,0,0


In [9]:
df.loc[df['is_spam']==1].iloc[10]['text_original']

'Vodi free reload ❤❤❤????????\n\nලේසියෙන්ම Reload ගන්න පුලුවන් ඇප් එකක් තමා vodi කියන්නෙ. කරන්න තියෙන්නෙ මෙච්චරයි play story යෙන් vodi app එක ගන්න register වෙද්දි referral code එකට මතක ඇතුව NTM479  ගහන්න. ඊට පස්සෙ මට මැසේජ් එකක් දාන්න referral එක ගහල register උනු අය. points 40 අරන් දෙන්නම් මන් ඊට පස්සෙ.code එක ගැහුවේ නැත්තම් එන් නැ හොදේ.\nDent වගේ invite කරන්න ඕන් නැ points හොයන්න.Ad තියෙනව බලන්න.ඒවයින් පොඉන්ට් හොයන්න පුලුවන්. \nAdd උනු අයට call අරන් points හොයන්නත් පුලුවන් ලේසියෙන්ම (ඇඩ් උනු අයගේ group හදල තියෙන්නෙ ඒකට) .... call ගත්තට සල්ලි කැපෙන් නෑ \nCall එක තප්.3 වගේ තියල කට් කරාම හරි...එක call එකකට points 3 ගානෙ එනවා.\nඑහෙම එකතු කරල reload ගන්න පුලුවන් \u2063ලේසියෙන්ම...\n\n????????????????????????????????????????????????????????????????????????????????????\nDead heart'

In [10]:
X = df[['has_link', 'has_subscribe', 'has_email', 'has_phone']]
y = df['is_spam']
X.head()

,has_link,has_subscribe,has_email,has_phone
0,1,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0


In [11]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=4)
gnb = GaussianNB()
gnb.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [12]:
pred = gnb.predict(x_test)
pred

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0], dtype=int64)

In [13]:
actual = np.array(y_test)
actual

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0], dtype=int64)

In [14]:
correct_guesses = 0;
for i in range(0, len(pred)):
    if actual[i] == pred[i]:
        correct_guesses += 1

correct_guesses

172

In [15]:
len(pred)

179

In [16]:
accuracy = 172/179.0
accuracy

0.9608938547486033

In [17]:
accuracy_score(y_test, pred)

0.9608938547486033